# Homework 1

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

# Context

For use in these problems, consider the data below, discussed in Veronesi's *Fixed Income Securities* Chapters 9, 10.
* interest-rate tree
* current term structure

In [54]:
import numpy as np
import pandas as pd
import sys

sys.path.insert(0, '../cmds')
from binomial import *

In [55]:
rate_tree = pd.DataFrame({'0':[.0174,np.nan],'0.5':[.0339,.0095]})
rate_tree.columns.name = 'time $t$'
rate_tree.index.name = 'node'
rate_tree.style.format('{:.2%}',na_rep='')

time $t$,0,0.5
node,,
0,1.74%,3.39%
1,,0.95%


The "tree" is displayed as a pandas dataframe, so it does not list "up" and "down" for the rows but rather an index of nodes. The meaning should be clear.

In [56]:
term_struct = pd.DataFrame({'maturity':[.5,1,1.5],'price':[99.1338,97.8925,96.1462]})
term_struct['continuous ytm'] = -np.log(term_struct['price']/100) / term_struct['maturity']
term_struct.set_index('maturity',inplace=True)
term_struct.style.format({'price':'{:.4f}','continuous ytm':'{:.2%}'}).format_index('{:.1f}')

,price,continuous ytm
maturity,,
0.5,99.1338,1.74%
1.0,97.8925,2.13%
1.5,96.1462,2.62%


This is the current term-structure observed at $t=0$.

# 1. Pricing a Swap

### 1.1 
Calculate the tree of bond prices for the 2-period, $T=1$, bond.

In [57]:
bond_tree = pd.DataFrame(index=rate_tree.index, columns=rate_tree.columns)
bond_tree.iloc[0, 0] = term_struct.loc[1, 'price']
bond_tree['0.5'] = np.exp(-rate_tree['0.5']*0.5)*100
bond_tree

time $t$,0,0.5
node,,
0,97.8925,98.319284
1,NaN,99.526126


### 1.2 
What is the risk-neutral probability of an upward movement of interest rates at $t=.5$?

In [58]:
A = np.exp(rate_tree.iloc[0,0]*0.5)
Z = bond_tree.loc[0,'0']
D = bond_tree.loc[1,'0.5']
U = bond_tree.loc[0,'0.5']

pstar = (A*Z-D)/(U-D)
pstar

0.6448615964742744

## The option contract

Consider a single-period swap that pays at time period 1 ($t=0.5$), the expiration payoff (and thus terminal value) is
* Payoff = $\frac{100}{2}(r_1 −c)$
* with $c=2\%$
* payments are semiannual

Take the viewpoint of a fixed-rate payer, floating rate receiver.

### 1.3 
What is the replicating trade using the two bonds (period 1 and period 2)?

In [59]:
replicating_port(term_struct.loc[[.5,1],'price'].values, bond_tree, swap_tree, dt='0.5')

,positions,value
cash,1.000862,99.13380
under,-1.010903,97.89250
derivative,NaN,0.25949


### 1.4 
What is the price of the swap?

In [60]:
payoff = lambda r: (100/2)*(r-0.02)
swap_tree = pd.DataFrame(index=rate_tree.index, columns=rate_tree.columns)
swap_tree['0.5'] = payoff(rate_tree['0.5'])
swap_tree.loc[0,'0'] = np.exp(-rate_tree.iloc[0,0] * 0.5) * np.array([pstar,1-pstar]) @ swap_tree['0.5'].values
swap_tree

time $t$,0,0.5
node,,
0,0.259464,0.695
1,NaN,-0.525


# 2. Using the Swap as the Underlying
As in the note, W.1, consider pricing the followign interest-rate option,
* Payoff is $100\max(r_K-r_1,0)$
* strike is $r_K$ is 2\%
* expires at period 1, ($t=0.5$) 

Unlike the note, price it with the swap used as the underlying, not the two-period ($t=1$) bond. You will once again use the period-1 ($t=0.5$) bond as the cash account for the no-arbitrage pricing.

So instead of replicating the option with the two treasuries, now you're replicating/pricing it with a one-period bond and two-period swap.

### 2.1
Display the tree of swap prices.

In [62]:
swap_tree

time $t$,0,0.5
node,,
0,0.259464,0.695
1,NaN,-0.525


### 2.2
What is the risk-neutral probability of an upward movement at $t=.5$ implied by the underlying swap tree? 

Is this the same as the risk-neutral probability we found when the bond was used as the underlying?

In [63]:
A = np.exp(rate_tree.iloc[0,0]*0.5)
Z = swap_tree.loc[0,'0']
D = swap_tree.loc[1,'0.5']
U = swap_tree.loc[0,'0.5']

pstar = (A*Z-D)/(U-D)
pstar

0.6448615964742744

We obtained the same risk-neutral probability using the swap tree as the underlying as the bond as the underlying

### 2.3
What is the price of the rate option? Is it the same as we calculated in the note, W.1.?

In [65]:
payoff = lambda r: 100 * np.maximum(0.02 - r,0)

floorlet_tree = pd.DataFrame(columns=rate_tree.columns, index=rate_tree.index)
floorlet_tree['0.5'] = payoff(rate_tree['0.5'])
floorlet_tree.loc[0,'0'] = np.exp(-rate_tree.iloc[0,0] * 0.5) * np.array([pstar, 1-pstar])@floorlet_tree['0.5']
floorlet_tree

time $t$,0,0.5
node,,
0,0.369665,0.00
1,NaN,1.05


The price of 0.369 mathces the rpice calcualted in W.1

# 3. Pricing a Call on a Bond

Try using the same tree to price a call on the period-2 bond, (1-year), at period 1 (6-months).
* Payoff = $\max(P_{1|2}-K,0)$
* Strike = \$99.00

### 3.1 
What is the replicating trade using the two bonds (period 1 and period 2) as above? (That is, we are no longer using the swap as the underlying.)

In [83]:
payoff2 = lambda p: np.maximum(p - 99,0)
rate_tree.columns = [float(i) for i in rate_tree.columns]
pstars = estimate_pstar(term_struct.loc[:rate_tree.index[-1],['price']],rate_tree)
pstars

0.0    0.644862
Name: pstar, dtype: float64

In [84]:
bond_tree.columns = [float(i) for i in bond_tree.columns]
derivtree = bintree_pricing(payoff2, rate_tree, undertree=bond_tree, pstars=pstars)
derivtree

,0.0,0.5
node,,
0,0.185229,0.000000
1,NaN,0.526126


In [85]:
replicating_port(term_struct.loc[[.5,1],'price'].values, bond_tree, derivtree, dt=.5)

,positions,value
cash,-0.428626,99.133800
under,0.435953,97.892500
derivative,NaN,0.185218


### 3.2 
What is the price of the European call option? 
* expiring at $T=.5$ 
* written on the bond maturing in 2 periods, ($t=1$)

In [86]:
derivtree.loc[0,0]

0.18522914483722105

# 4 Two-Period Tree

Consider an expanded, **2 period** tree. (Two periods of uncertainty, so with the starting point, three periods total.)

In [61]:
new_col = pd.Series([.05,.0256,.0011],name='1')
rate_tree_multi = pd.concat([rate_tree,new_col],ignore_index=True,axis=1)
rate_tree_multi.columns = pd.Series(['0','0.5','1'],name='time $t$')
rate_tree_multi.index.name = 'node'
rate_tree_multi.style.format('{:.2%}',na_rep='')

time $t$,0,0.5,1
node,,,
0,1.74%,3.39%,5.00%
1,,0.95%,2.56%
2,,,0.11%


### 4.1

Calculate and show the tree of prices for the 3-period bond, $T=1.5$.

In [90]:
payoff3 = lambda r: payoff_bond(r, 0.5)
pstars = estimate_pstar(term_struct,rate_tree_multi)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

### 4.2
Report the risk-neutral probability of an up movement at $t=1$.

The risk-neutral probability of an up movement at $t=0.5$ continues to be as you calculated in 2.3.

### 4.3
Calculate the price of the European **call** option?
* expiring at $T=1$ 
* written on the bond maturing in 3 periods, ($t=1.5$)

### 4.4
Consider a finer time grid. Let $dt$ in the tree now be 1/30 instead of 0.5.

Using this smaller time step, compute the $t=0$ price of the following option:
* option expires at $t=1$
* written on bond maturing at $t=1.5

# 5 American Style
### 5.1
Use the two-period tree from part 4, but this time to price an American-style **put** option.

Use a grid of $dt=.5$.
* What is its value at $t=0$?
* Which nodes would you exercise it early?

### 5.2
Change the grid to $dt=1/30$, as in 4.4. 
* What is its value at $t=0$?
* Make a visualization showing which nodes have early exercise. (I suggest using a dataframe and the `heatmap` from `seaborn`.